In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('Data/pitchesv2.csv')
df.head()

/Users/joshuarkarnad/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,uid,game_pk,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,pcount_pitcher,balls,...,nasty,spin_dir,spin_rate,cc,on_1b,on_2b,on_3b,right_side_plate,right_handed_pitcher,batter_height_inches
0,14143252,286874,118,108,1,0,7,1,1,0,...,47.0,183.148,2519.455,NaN,0,0,0,1,1,70
1,14143253,286874,118,108,1,0,7,2,2,0,...,25.0,187.663,2838.803,NaN,0,0,0,1,1,70
2,14143254,286874,118,108,1,0,7,3,3,1,...,64.0,179.643,2701.919,NaN,0,0,0,1,1,70
3,14143255,286874,118,108,1,0,7,4,4,2,...,49.0,184.623,2683.280,NaN,0,0,0,1,1,70
4,14143256,286874,118,108,1,0,7,5,5,2,...,32.0,182.338,3352.205,NaN,0,0,0,1,1,70


In [3]:
# Dropping these variables to make it easier on my computer but 
# I would do previous events for all of these variables
# Kept the most important variables based on my baseball intuition
previous_pitch_columns = ['type', 'x', 'y', 'pitch_type', 'start_speed', 'event']
df = df.drop(['final_balls', 'final_strikes', 'event4', 'is_final_pitch', 'pitch_des', 
              'end_speed', 'pfx_x', 'pfx_z', 'px', 'pz', 'x0', 'z0', 'y0', 'vz0', 
              'vy0', 'az', 'ay', 'break_length', 'break_angle', 'break_y', 
              'spin_dir', 'spin_rate', 'nasty', 'vx0', 'ax', 'sz_top', 'sz_bot',
             'event2', 'event3', 'cc'], axis=1)

In [5]:
#Creating Previous Pitch Features
pitching_groupby = ['pitcher_id', 'at_bat_num', 'batter_id', 'game_pk', 'inning']
df['previous_pitch'] = df.groupby(pitching_groupby)['pitch_type'].shift(1)
df['previous_pitch'] = df['previous_pitch'].fillna('First_Pitch')

df['previous_type'] = df.groupby(pitching_groupby)['type'].shift(1)
df['previous_type'] = df['previous_type'].fillna('First_Pitch')
df = df.drop('type', axis = 1)


# Changing More Previous Pitch features but not using the prefix previous
previous_pitch_columns = ['x', 'y', 'start_speed', 'zone']

for column in previous_pitch_columns:
    df[column] = df.groupby(pitching_groupby)[column].shift(1)
    df[column] = df[column].fillna(df[column].mean())

In [6]:
# Creating Previous At Bat Features
at_bat_groupby = ['pitcher_id', 'inning','game_pk']
df['previous_at_bat_event'] = df.groupby(at_bat_groupby)['event'].shift(1)
df['previous_at_bat_event'] = df['previous_at_bat_event'].fillna('New_Inning')
df['previous_score'] = df.groupby(at_bat_groupby)['score'].shift(1)
df['previous_score'] = df['previous_score'].fillna('New_Inning')

In [8]:
# Dropping event and score because I've created previous event and previous score
df = df.drop(['event', 'score'], axis = 1)
df.to_csv('Data/pitchesv3.csv', index = False)